## Self-Supervised Pre-training [30 marks]

### What is rotation classification task?
One important self-training task is the task of rotation classification. Here given a set of unlabeled images, we randomly rotate it to either of the following angles {0, 90, 180, 270} degrees and train a image rotation classifier which predicts by what angle the initial image has been rotated to generate the current image. The problem is set up as a classification problem since we only rotate our images by a fixed set of angles as mentioned before and the corresponding ground truth labels being {0, 1, 2, 3}.

<center>
<img src="./fig/Self-training-rot.png" width="524" height="300">
</center>

Once the self-training based pretraining is done, we strip away the final classification layer(which is a linear layer) and add Convolutional or linear layers as per the downstream task's requirement.


### Details of this problem statement.

1. Take the CIFAR-10 dataset, each class has 5000 samples and there are 10 classes.
Split the dataset in 2 parts (A) 40000 and (B) 10000 each with equal number of samples per class in each split.
2. Discard the labels of the samples in the first set.
3. Take a resnet-18 (initialised) and strip the imagenet classification layer with a 4 way classification layer.
4. Train this network on the self-training task of classifiying the rotation of the image.
5. Once this self-supervised pretraining is done, strip the classification layer and add a classification layer for CIFAR-10 classification this is finetuned on the set B for the task of image classification.
6. Log the loss(cross entropy) and accuracies for both the pre-training task and classfication task.

You are free tou use ML API of your choice. Your work will be checked for plagiarism!!

### Citations and helpful references
[1]. https://www.youtube.com/watch?v=8L10w1KoOU8&t=694s <br>
[2]. https://openaccess.thecvf.com/content_CVPR_2019/papers/Feng_Self-Supervised_Representation_Learning_by_Rotation_Feature_Decoupling_CVPR_2019_paper.pdf

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary
from torchvision import datasets, models
from torch.utils.data import DataLoader, Subset, random_split
from tqdm import tqdm
import copy

In [2]:
# setting fixed seed for reproducibility
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Defining CIFAR-10 data transformations
transform = transforms.ToTensor()

Using device: cuda


In [3]:
class RotationDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self, index):
        img, _ = self.dataset[index]
        angle = torch.randint(0, 4, (1,), dtype=torch.long).item()
        img = transforms.functional.rotate(img, angle*90)
        if self.transform:
            img = self.transform(img)
        return img, angle

    def __len__(self):
        return len(self.dataset)

In [4]:
train_set_A = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_dataset_A = RotationDataset(train_set_A)
train_loader_A = DataLoader(train_dataset_A, batch_size=64, shuffle=True, num_workers=2)

test_set_B = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())
test_loader_B = DataLoader(test_set_B, batch_size=64, shuffle=False, num_workers=2)


100%|██████████| 170498071/170498071 [00:04<00:00, 39215240.91it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
class RotationClassifier(nn.Module):
    def __init__(self, num_classes=4):
        super(RotationClassifier, self).__init__()
        resnet = models.resnet18(pretrained=True)
        # Remove the final classification layer
        self.features = nn.Sequential(*list(resnet.children())[:-1])
        # Add a new 4-way classification layer for rotation
        self.rotation_classifier = nn.Linear(resnet.fc.in_features, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.rotation_classifier(x)
        return x

In [6]:
def train_rotation_classifier(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')


In [7]:
def finetune_classification(model, train_loader, criterion, optimizer, epochs=5):
    # Remove rotation classifier
    # Add a new classification layer for CIFAR-10
    model.rotation_classifier = nn.Linear(model.rotation_classifier.in_features, 10)

    model = model.to(device)

    model.train()
    for epoch in range(epochs):
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')



In [8]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f'Test Accuracy: {accuracy * 100:.2f}%')


In [9]:
model = RotationClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer_rotation = optim.Adam(model.parameters(), lr=0.001)
optimizer_finetune = optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 121MB/s]


In [10]:
# Train the rotation classifier
train_rotation_classifier(model, train_loader_A, criterion, optimizer_rotation)

Epoch 1/10, Loss: 1.2572190761566162
Epoch 2/10, Loss: 0.8470165133476257
Epoch 3/10, Loss: 1.2240005731582642
Epoch 4/10, Loss: 0.23080208897590637
Epoch 5/10, Loss: 0.618952751159668
Epoch 6/10, Loss: 0.6897918581962585
Epoch 7/10, Loss: 0.21911293268203735
Epoch 8/10, Loss: 0.5494576692581177
Epoch 9/10, Loss: 0.2722645103931427
Epoch 10/10, Loss: 0.36184564232826233


In [11]:
# Evaluate the model on the downstream classification task
evaluate_model(model, train_loader_A)

Test Accuracy: 85.01%


In [12]:
finetune_model = copy.deepcopy(model)
# Fine-tune on CIFAR-10 classification task
finetune_classification(finetune_model, test_loader_B, criterion, optimizer_finetune, 10)

Epoch 1/10, Loss: 2.181859254837036
Epoch 2/10, Loss: 2.181859254837036
Epoch 3/10, Loss: 2.181859254837036
Epoch 4/10, Loss: 2.181859254837036
Epoch 5/10, Loss: 2.181859254837036
Epoch 6/10, Loss: 2.181859254837036
Epoch 7/10, Loss: 2.181859254837036
Epoch 8/10, Loss: 2.181859254837036
Epoch 9/10, Loss: 2.181859254837036
Epoch 10/10, Loss: 2.181859254837036


In [13]:
# Evaluate the model on the downstream classification task
evaluate_model(finetune_model, test_loader_B)

Test Accuracy: 9.66%


#Results
Accuracy on rotation task with dataset A : 85.01%

Accuracy on classification task with dataset B : 9.66%


### Rough work

In [14]:
torch.save(model.state_dict(), '/content/model_first')

In [24]:
summary(model,( 3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]          36,864
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
       BasicBlock-11             [-1, 64, 8, 8]               0
           Conv2d-12             [-1, 64, 8, 8]          36,864
      BatchNorm2d-13             [-1, 64, 8, 8]             128
             ReLU-14             [-1, 6

In [ ]:
summary(finetune_model,( 3, 32, 32))

In [32]:
resnet = resnet18(pretrained=True)
        # Remove the classification layer
# resnet = nn.Sequential(*list(resnet.children())[:-1])
# self.resnet = nn.Sequential(*resnet)
# self.fc = nn.Linear(self.resnet.fc.in_features, 4)  # 4-wa

summary(resnet,( 3, 32, 32))
# print(resnet,(3,))

In [15]:
resnet = models.resnet18(pretrained=True)

In [16]:
resnet.fc.in_features

512

In [25]:
model.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [30]:
model.rotation_classifier.in_features

512